# Combining COMTRADE and Flaring data

GGFR have a global flare datasets derived from Elvidge at point locations. They further map these point locations to operators via oil field mapping. This means there are a number of datasets:

1. Point locations of flaring
2. Mapped oil field locations (from Woods Mackenzie)
3. Tabulated national accounts (of flaring)

Harshit suggested there could be an interesting combination of the COMTRADE data to see what the comparison of nationally generated flaring vs imported flaring.

Flaring Supply Index (FSI) is a concept proposed for an indicator that would be able to describe and compare the ’flaring footprint’ of different oil supply corridors from their source to the destination country’s border. The formula(s) for such an index may be described as follows.
Flare Supply Index of a Country X= (∑_(i=1)^n▒〖Crude Import by X from Country Yi * Flare Intensity of Country Yi〗)/(Total Crude Imported by Country X)


## Definitions
Flare Intensity - ???  
TOE - Tonnes of oil equivalent

## TO DO
1. TOE is not the appropriate flaring comparison for intensity, figure out what it is
2. Right now, the intensity is taken just from 2019; should use yearly data

In [1]:
import sys, os, importlib
import json, geojson, pycountry

import pandas as pd
import geopandas as gpd
import numpy as np

In [55]:
# Define input variables
# COMTRADE data can be downloaded here - https://datacatalog.worldbank.org/dataset/global-comtrade-flows-data
comtrade_file = '/home/wb411133/data/Projects/INFRA/FLOWS/Oil_CRUDE_ONLY/GEOJSON/country_flows_imports.geojson'
flare_data = '/home/public/Data/GLOBAL/INFRA/FLARING/2019_flare_catalog.csv'
flare_national_data = '/home/public/Data/GLOBAL/INFRA/FLARING/National_flaring_2019.csv'
out_folder = "/home/wb411133/data/Projects/INFRA/FLARING/Data"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

inCom = gpd.read_file(comtrade_file)
inFlare = pd.read_csv(flare_data)
inF = pd.read_csv(flare_national_data)

The location specific flare data is not necessary for calculating the FSI, but it could be used in other calculations

In [49]:
inCom.loc[(inCom['Reporter ISO'] == "JPN") & (inCom['Year'] == '2017')].sort_values('TOE', ascending=False)

,Reporter ISO,Partner ISO,Trade Flow,Year,Qty,Trade Value (US$),TOE,Reporter_Pt,Partner_Pt,Value per unit,geometry
1390,JPN,SAU,Import,2017,6.342730e+10,2.563010e+10,7.759041e+07,POINT (137.9632416223659 37.54022267074495),POINT (44.54372307264682 24.12390168405157),0.404086,LINESTRING (44.54372307264682 24.1239016840515...
1357,JPN,ARE,Import,2017,3.770237e+10,1.562288e+10,4.612118e+07,POINT (137.9632416223659 37.54022267074495),POINT (54.32869156271079 23.90682311096173),0.414374,LINESTRING (54.32869156271079 23.9068231109617...
1388,JPN,QAT,Import,2017,1.154190e+10,4.690048e+09,1.411916e+07,POINT (137.9632416223659 37.54022267074495),POINT (51.19868832434384 25.28053193101709),0.406350,LINESTRING (51.19868832434384 25.2805319310170...
1379,JPN,KWT,Import,2017,1.153480e+10,4.394215e+09,1.411048e+07,POINT (137.9632416223659 37.54022267074495),POINT (47.61290724370362 29.35199533223659),0.380953,LINESTRING (47.61290724370362 29.3519953322365...
1389,JPN,RUS,Import,2017,9.007100e+09,3.709188e+09,1.101836e+07,POINT (137.9632416223659 37.54022267074495),POINT (96.59005059203922 61.95356043548515),0.411807,LINESTRING (96.59005059203922 61.9535604354851...
1376,JPN,IRN,Import,2017,8.711050e+09,3.506940e+09,1.065620e+07,POINT (137.9632416223659 37.54022267074495),POINT (54.19893299230245 32.73732737127096),0.402585,LINESTRING (54.19893299230245 32.7373273712709...
1377,JPN,IRQ,Import,2017,2.704910e+09,1.008425e+09,3.308907e+06,POINT (137.9632416223659 37.54022267074495),POINT (43.77473247071985 33.04588584276406),0.372813,LINESTRING (43.77473247071985 33.0458858427640...
1381,JPN,MEX,Import,2017,2.233370e+09,7.816628e+08,2.732074e+06,POINT (137.9632416223659 37.54022267074495),POINT (-102.5326675060585 23.93818198998856),0.349993,LINESTRING (-102.5326675060585 23.938181989988...
1375,JPN,IDN,Import,2017,1.914630e+09,7.780263e+08,2.342160e+06,POINT (137.9632416223659 37.54022267074495),POINT (117.2772517784194 -2.226238666571241),0.406359,LINESTRING (117.2772517784194 -2.2262386665712...
1385,JPN,OMN,Import,2017,1.581370e+09,5.972156e+08,1.934484e+06,POINT (137.9632416223659 37.54022267074495),POINT (56.10260498874897 20.5968629472638),0.377657,LINESTRING (56.10260498874897 20.5968629472638...


In [50]:
# Limit the flaring summaries to 2019
flare_2019 = inF.filter(regex="2019")
flare_2019.columns = ["Volume","Intensity"]
flare_2019['country'] = inF['country']
flare_2019.head()

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Volume,Intensity,country
0,23212.170060,5.862699884,Russia
1,17914.219000,10.37717386,Iraq
2,17293.742520,3.868390139,United States
3,13781.155540,12.59168227,Iran
4,9541.420483,29.81330551,Venezuela


In [51]:
def get_country(x):
    ''' Convert country name to ISO3
    
    :param: x [string] - name of country to convert
    :returns: [string] - ISO3 code of country
    '''
    try:
        res = pycountry.countries.search_fuzzy(x)
        return(res[0].alpha_3)
    except:
        return("NA")

#xx = get_country("Canada")
flare_2019['ISO3'] = flare_2019['country'].apply(lambda x: get_country(x))
flare_2019.head()

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Volume,Intensity,country,ISO3
0,23212.170060,5.862699884,Russia,RUS
1,17914.219000,10.37717386,Iraq,IRQ
2,17293.742520,3.868390139,United States,USA
3,13781.155540,12.59168227,Iran,IRN
4,9541.420483,29.81330551,Venezuela,VEN


In [52]:
# 5 of the country names don't grok, set them manually

#flare_2019.loc[flare_2019['ISO3'] == "NA"]
flare_2019.iloc[16, 3] = "COG"
flare_2019.iloc[30, 3] = "ARE"
flare_2019.iloc[44, 3] = "COD"
flare_2019.iloc[60, 3] = "NER"
flare_2019.loc[flare_2019['ISO3'] == "NA"]

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Volume,Intensity,country,ISO3


In [53]:
#Combine the flare intensity information with the comtrade data
in_flaring_combined = pd.merge(inCom, flare_2019, left_on="Partner ISO", right_on="ISO3", how="left")
#Drop unnecessary columns
in_flaring_combined = in_flaring_combined.drop(['Reporter_Pt', 'Partner_Pt', 'geometry', 'country', 'ISO3'], axis=1)

def tryFloat(x):
    try:
        return(float(x))
    except:
        return(0.)

# Multiply the TOE from the comtrade data by the intensity information
### TODO: TOE is not the unit appropriate for intensity calculations; however, the relative rank is still useful
in_flaring_combined['TOE'] = in_flaring_combined['TOE'].apply(float)
in_flaring_combined['Intensity'] = in_flaring_combined['Intensity'].apply(tryFloat)
in_flaring_combined['TOE_I'] = in_flaring_combined['TOE'] * in_flaring_combined['Intensity']
in_flaring_combined.head()

,Reporter ISO,Partner ISO,Trade Flow,Year,Qty,Trade Value (US$),TOE,Value per unit,Volume,Intensity,TOE_I
0,,AGO,Import,2017,3.528180e+09,1.466747e+09,4316011.000,0.415723,2325.103979,4.400772,1.899378e+07
1,,ARE,Import,2017,4.194376e+09,1.653489e+09,5130966.000,0.394216,900.272261,0.721855,3.703813e+06
2,,AUS,Import,2015,8.098200e+07,2.589667e+07,99065.016,0.319783,1389.791288,10.554468,1.045579e+06
3,,AZE,Import,2017,1.666776e+09,7.096852e+08,2038961.500,0.425783,222.731902,0.795309,1.621605e+06
4,,BRA,Import,2017,1.633857e+09,6.426855e+08,1998692.000,0.393355,1135.508776,1.115980,2.230501e+06


In [54]:
in_flaring_combined.loc[(in_flaring_combined['Reporter ISO'] == "JPN") & (in_flaring_combined['Year'] == "2017")].sort_values('TOE', ascending=False).head()

,Reporter ISO,Partner ISO,Trade Flow,Year,Qty,Trade Value (US$),TOE,Value per unit,Volume,Intensity,TOE_I
1390,JPN,SAU,Import,2017,6.342730e+10,2.563010e+10,77590408.0,0.404086,2100.455544,0.585661,4.544171e+07
1357,JPN,ARE,Import,2017,3.770237e+10,1.562288e+10,46121184.0,0.414374,900.272261,0.721855,3.329281e+07
1388,JPN,QAT,Import,2017,1.154190e+10,4.690048e+09,14119165.0,0.406350,1343.748416,2.422041,3.419720e+07
1379,JPN,KWT,Import,2017,1.153480e+10,4.394215e+09,14110483.0,0.380953,730.990357,0.692833,9.776203e+06
1389,JPN,RUS,Import,2017,9.007100e+09,3.709188e+09,11018356.0,0.411807,23212.170060,5.862700,6.459731e+07


In [56]:
in_flaring_combined.to_csv(os.path.join(out_folder, "FLARING_TRADE_CRUDEONLY_COMBINED_ALLTRADE.csv"))

In [57]:
# Aggregate imported TOE, Value and TOE by import intensity
agg = {'TOE':'sum','Trade Value (US$)':'sum', 'Intensity':'mean','TOE_I':'sum'}
res = in_flaring_combined.groupby(['Reporter ISO', "Year"]).aggregate(agg)
res = res.reset_index()
res.head(20)

,Reporter ISO,Year,TOE,Trade Value (US$),Intensity,TOE_I
0,,2010,7.372645e+04,2.772806e+07,3.101707,2.286779e+05
1,,2012,4.905578e+05,3.360215e+08,7.054142,3.460464e+06
2,,2015,5.743443e+05,1.844477e+08,4.879513,2.862283e+06
3,,2016,1.588106e+05,3.999923e+07,13.914510,2.209772e+06
4,,2017,5.311106e+07,1.690616e+10,5.426175,1.561046e+08
5,AFG,2011,1.208880e+03,3.162290e+05,4.083499,6.824355e+03
6,AGO,2010,3.669890e-03,1.300000e+01,13.914510,5.106472e-02
7,AGO,2011,1.223297e-03,6.000000e+00,0.037056,4.533062e-05
8,AGO,2013,1.198831e-01,8.820000e+02,1.052053,3.805322e-02
9,AGO,2015,8.807736e-02,1.812100e+04,0.123234,0.000000e+00
